In [1]:

import os

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet
from azure.identity import DefaultAzureCredential

# user_functions.py で定義した関数セットを読み込む
from user_functions import get_user_functions

# Azure AI Foundry Project の接続文字列を環境変数から取得
if "PROJECT_CONNECTION_STRING" not in os.environ:
    os.environ["PROJECT_CONNECTION_STRING"] = ""

conn_str = os.environ["PROJECT_CONNECTION_STRING"]
credential = DefaultAzureCredential()

# クライアント生成
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=conn_str
)


In [2]:
user_message = "東京から行けるおすすめの温泉地を教えて。ユニークな視点とコスパ重視の両方で比較したいです。"

In [ ]:
# エージェントの作成
agentB = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="UniqueAgent",
    instructions="あなたはユニークな視点を持つアシスタントです。一般的ではない特別な視点から回答してください。",
)
print(f"AgentB created: {agentB.id}")

agentC = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="CostEffectiveAgent",
    instructions="あなたはコスパ重視のアシスタントです。コストパフォーマンスを重視して回答してください。",
)
print(f"AgentC created: {agentC.id}")


In [ ]:
# ツールの定義
funcs = get_user_functions(project_client, agentB.id, agentC.id)
functions = FunctionTool(funcs)  

toolset = ToolSet()
toolset.add(functions)

# エージェント（オーケストレーター）を作成
agentA = project_client.agents.create_agent(
    model="gpt-4o-mini", 
    name="OrchestratorAgent",
    instructions=(
        "あなたはオーケストレーターとして、"
        "UniqueAgent と CostEffectiveAgent のツールを必要に応じて呼び出し、それぞれの回答を統合してください。"
    ),
    toolset=toolset
)
print(f"Agent created, ID: {agentA.id}")


In [ ]:
# スレッドの作成とメッセージ送信
thread = project_client.agents.create_thread()
print(f"Thread created, ID: {thread.id}")

msg = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_message
)
print(f"User message created: {msg.id}")


In [ ]:
# エージェントに実行を依頼する。また関数もここで実行
runA = project_client.agents.create_and_process_run(
    thread_id=thread.id,
    assistant_id=agentA.id
)

print(f"Run finished with status: {runA.status}")

if runA.status == "failed":
    print(f"Run failed: {runA.last_error}")


In [ ]:
# 7) 結果メッセージを取得して表示
messages_json = project_client.agents.list_messages(thread_id=thread.id)
if not messages_json.get("data"):
    print("No messages returned.")
else:
    data = messages_json["data"]
    for i, m in enumerate(data, start=1):
        role = m["role"]
        text_list = []
        for c in m.get("content", []):
            if c.get("type") == "text":
                text_list.append(c["text"]["value"])
        joined = "\n".join(text_list)
        print(f"[Message {i} | {role}]\n{joined}\n")

In [ ]:
# (8) 後片付け (エージェント & スレッド削除)
project_client.agents.delete_agent(agentA.id)
print(f"Deleted AgentA: {agentA.id}")

project_client.agents.delete_agent(agentB.id)
print(f"Deleted AgentB: {agentB.id}")

project_client.agents.delete_agent(agentC.id)
print(f"Deleted AgentC: {agentC.id}")

project_client.agents.delete_thread(thread.id)
print(f"Deleted thread: {thread.id}")
